## Model Design using Pytorch 
This is a summary of building Neural network as a template for helping new users of pytroch.
You can implement this for your project. 
All of the result just for experimentation. You can change based on your projects requirement. 

In [12]:
import torch.nn as nn 
import torch.nn.functional as F 
import torch 
from torch import optim

In [5]:
class SimpleNet(nn.Module):
    ## created layers as classattributes
    def __init__(self):
        ## call the base class to initialize params
        super(SimpleNet,self).__init__()
        self.fc1 = nn.Linear(2048,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,2)
    
    ## Required to define how the model process the parameters
    def forward(self,x):
        x = x.view(-1,2048)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x),dim=1)
        return x 
        

In [7]:
simplenet = SimpleNet()
print(simplenet)

SimpleNet(
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)


nn.Module also supports for CNN, Dropout, and BatchNomarlization to implement in our model. 

## Training Loop
implemented training loop using LeNet5 model 


In [11]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,int(x.nelement()/x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

device = ("cuda" if torch.cuda.is_available() else "cpu")
##move to GPU 
model = LeNet5().to(device=device)

In [15]:
from torchvision.datasets import CIFAR10
train_data = CIFAR10(root="./train/",train=True,download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./train/cifar-10-python.tar.gz to ./train/


In [16]:
test_data = CIFAR10(root="./test/",train=False,download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./test/cifar-10-python.tar.gz to ./test/


## Transform the data 

In [19]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = (0.4914,0.4822,0.4465),
        std = (0.2023,0.1994,0.2010))])

train_data = CIFAR10(root="./train/",train=True,
                     download=True,transform=train_transforms)


Files already downloaded and verified


In [21]:
from torchvision import transforms

test_transforms = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = (0.4914,0.4822,0.4465),
        std = (0.2023,0.1994,0.2010))])

test_data = CIFAR10(root="./test/",train=False,
                     download=True,transform=test_transforms)


Files already downloaded and verified


## Data Batching using DataLoader

In [22]:
trainloader =torch.utils.data.DataLoader(train_data,
                                         batch_size=16,
                                         shuffle=True)

In [23]:
testloader  =torch.utils.data.DataLoader(test_data,
                                         batch_size=16,
                                         shuffle=False)

In [25]:
## optimization 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)

N_EPOCHS = 10
for epoch in range(1,N_EPOCHS+1):
    epoch_loss = 0.0
    
    for inputs,labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss +=loss.item()
    print(f"Epoch {epoch} Loss {epoch_loss/len(trainloader)}")
        

Epoch 1 Loss 1.8207738812637329
Epoch 2 Loss 1.6673013194084167
Epoch 3 Loss 1.611757234916687
Epoch 4 Loss 1.588782156829834
Epoch 5 Loss 1.5781982357215882
Epoch 6 Loss 1.575931268749237
Epoch 7 Loss 1.566828318157196
Epoch 8 Loss 1.5608065349769593
Epoch 9 Loss 1.5636185508155822
Epoch 10 Loss 1.5685766509628296


## Validation 

In [27]:
from torch.utils.data import random_split 
train_set,val_set = random_split(train_data,[40000,10000])

trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=16,
                                          shuffle=True)
valloader = torch.utils.data.DataLoader(val_set,
                                          batch_size=16,
                                          shuffle=True)

print(len(trainloader),len(valloader))

2500 625


In [29]:
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)

In [31]:
N_EPOCHS = 10
for epoch in range(1,N_EPOCHS+1):
    train_loss = 0.0
    model.train()
    for inputs,labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss +=loss.item()
        
        
    val_loss =0.0
    model.eval()
    for inputs,labels in valloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        val_loss +=loss.item()
    print(f"Epoch {epoch} Train Loss {epoch_loss/len(trainloader)} Val loss {val_loss/len(valloader)}")

Epoch 1 Train Loss 3.1996474461317064 Val loss 1.6014307223320008
Epoch 2 Train Loss 4.802808263874054 Val loss 1.6740261289596559
Epoch 3 Train Loss 6.413015327715874 Val loss 1.603927476119995
Epoch 4 Train Loss 8.018378995585442 Val loss 1.697635841178894
Epoch 5 Train Loss 9.63322612273693 Val loss 1.5807183632850648
Epoch 6 Train Loss 11.2521382843256 Val loss 1.5649494188308717
Epoch 7 Train Loss 12.86784440665245 Val loss 1.6468664051055908
Epoch 8 Train Loss 14.477429402422905 Val loss 1.6350692648887635
Epoch 9 Train Loss 16.108355063533782 Val loss 1.6147620951652526
Epoch 10 Train Loss 17.732579513716697 Val loss 1.674453691291809


Validation occurs at every epoch after the training has been processed 
. During validation, the model is passed data which has not seen before. Only forward pass during validation. 

## Testing

In [35]:
num_correct = 0.0 
for x_test_batch,y_test_batch in testloader:
    model.eval()
    
    y_test_batch = y_test_batch.to(device)
    x_test_batch = x_test_batch.to(device)
    y_pred_batch = model(x_test_batch)
    _,predicted = torch.max(y_pred_batch,1)
    num_correct +=(predicted==y_test_batch).float().sum()
    
accuracy =num_correct / (len(testloader)*testloader.batch_size)

print(f"Test Accuracy {accuracy}")

Test Accuracy 0.4056999981403351


## Saving Models

In [37]:
torch.save(model.state_dict(),"./lenet5_model.pt")
model = LeNet5().to(device)
model.load_state_dict(torch.load("./lenet5_model.pt"))

<All keys matched successfully>